In [1]:
from pulp import *

# Dados do problema
projetos = {
    1: {'ano0': 5, 'ano1': 4, 'ano2': 6, 'vpl': 22},
    2: {'ano0': 4, 'ano1': 3, 'ano2': 3, 'vpl': 17},
    3: {'ano0': 3, 'ano1': 2, 'ano2': 1, 'vpl': 16},
    4: {'ano0': 4, 'ano1': 1, 'ano2': 2, 'vpl': 14},
    5: {'ano0': 5, 'ano1': 3, 'ano2': 6, 'vpl': 20}
}

recursos_disponiveis = {'ano0': 12, 'ano1': 10, 'ano2': 9}

# Cria o problema de maximização
prob = LpProblem("Investiilia_Orcamento_Capital", LpMaximize)

# Variáveis de decisão (binárias: 1 se projeto for selecionado, 0 caso contrário)
x = LpVariable.dicts("Projeto", projetos.keys(), cat='Binary')

# Função objetivo: Maximizar o VPL total
prob += lpSum([projetos[j]['vpl'] * x[j] for j in projetos])

# Restrições de recursos por ano
prob += lpSum([projetos[j]['ano0'] * x[j] for j in projetos]) <= recursos_disponiveis['ano0'], "Recurso_Ano0"
prob += lpSum([projetos[j]['ano1'] * x[j] for j in projetos]) <= recursos_disponiveis['ano1'], "Recurso_Ano1"
prob += lpSum([projetos[j]['ano2'] * x[j] for j in projetos]) <= recursos_disponiveis['ano2'], "Recurso_Ano2"

# Resolve o problema
prob.solve()

# Exibe os resultados
print("Status:", LpStatus[prob.status])
print("VPL Total Máximo (R$ milhões):", value(prob.objective))
print("\nProjetos Selecionados:")
for j in projetos:
    if x[j].value() == 1:
        print(f"Projeto {j}: Investimento (ano0: {projetos[j]['ano0']}, ano1: {projetos[j]['ano1']}, ano2: {projetos[j]['ano2']}) - VPL: {projetos[j]['vpl']}")

print("\nRecursos Utilizados:")
print(f"Ano 0: {sum(projetos[j]['ano0'] * x[j].value() for j in projetos)}/{recursos_disponiveis['ano0']}")
print(f"Ano 1: {sum(projetos[j]['ano1'] * x[j].value() for j in projetos)}/{recursos_disponiveis['ano1']}")
print(f"Ano 2: {sum(projetos[j]['ano2'] * x[j].value() for j in projetos)}/{recursos_disponiveis['ano2']}")

Status: Optimal
VPL Total Máximo (R$ milhões): 52.0

Projetos Selecionados:
Projeto 1: Investimento (ano0: 5, ano1: 4, ano2: 6) - VPL: 22
Projeto 3: Investimento (ano0: 3, ano1: 2, ano2: 1) - VPL: 16
Projeto 4: Investimento (ano0: 4, ano1: 1, ano2: 2) - VPL: 14

Recursos Utilizados:
Ano 0: 12.0/12
Ano 1: 7.0/10
Ano 2: 9.0/9
